In [ ]:
using GLPK
using JuMP

In [53]:
lines = readlines("data/VA01.dat")
k = 3 #instance 1
n = parse(Int,lines[1]) 
ci = [parse(Int,x) for x in split(lines[2])]
cp = fill(0, n, n)
for i in 1:(n-1)
    currentLine = [parse(Int,x) for x in split(lines[2+i])]
    for j in 1:length(currentLine)
        cp[i,i+j] = currentLine[j]
        cp[i+j,i] = currentLine[j]
    end
end
p = [parse(Int,x) for x in split(lines[n + 5])]
P = fill(0.8*sum(p)/k,k)

3-element Vector{Float64}:
 688.5333333333333
 688.5333333333333
 688.5333333333333

In [2]:
m = Model();

In [4]:
@variable(m, ci[n], binary=true)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 20
And data, a 1-element Array{VariableRef,1}:
 ci[20]

In [5]:
@variable(m, p[n] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 20
And data, a 1-element Array{VariableRef,1}:
 p[20]

In [6]:
@variable(m, P[n] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 20
And data, a 1-element Array{VariableRef,1}:
 P[20]

In [7]:
@variable(m, cp[n, n], binary=true);

In [8]:
@variable(m, v[n, k], binary=true);

In [9]:
@objective(m, Max, sum(sum(ci[j]*v[i,j] for j in n) for i in k) + sum(sum(sum(cp[o,j] * (v[i,o]+v[i,j])/2 for o in n) for j in n) for i in k))

ci[20]*v[20,20] + cp[20,20]*v[20,20]

In [10]:
@constraint(m, [j in n], sum(v[i,j] for i in k) <= 1)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, 20
And data, a 1-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 v[20,20] ≤ 1.0

In [11]:
@constraint(m, [i in k], sum(p[j] * v[i,j] for j in n) <= P[i])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, 20
And data, a 1-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 p[20]*v[20,20] - P[20] ≤ 0.0